 # 기본 전처리 및 Baseline model

* 데이터에 대한 기본 전처리를 수행한 후
* Baseline 모델을 생성하고 평가합니다.

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst

import warnings
warnings.filterwarnings(action='ignore')

## (2) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/retail_demand2.csv'
data = pd.read_csv(path, usecols = ['date', 'sales', 'tot_sales', 'comp_sales'])
data = data.loc[data['date']<= '2015-10-31'].reset_index(drop = True)
data.head()

* 변수 설명

    * date : 날짜
    * sales : A유통회사 a 매장 aa상품의 일별 판매량
    * tot_sales : A유통회사 전 매장 aa상품의 일별 판매량
    * comp_sales : a매장 인근 B유통회사(경쟁사) b매장 aa상품의 일별 판매량

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

In [ ]:
temp = data[-100:]
plt.figure(figsize = (20,8))
plt.plot(temp['sales'], marker ='o')
plt.grid()
plt.show()

# 2.기본 전처리

## (1) 날짜 인덱스

### 1) 날짜 타입으로 변경하기

In [ ]:
# 데이터프레임의 정보를 살펴 봅시다.
data.info()

* 날짜 요소로 변환하기 : pd.to_datetime( *날짜형식으로 저장된 문자열 변수*   , **format** = )  
* format : 일반적인(쉽게 인식 가능한 형태)는 생략 가능 (예 : yyyy-mm-dd hh:mi:ss)
    * to_datetime : https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
    * format : https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.info()

### 2) 날짜를 인덱스로 변환하기

In [ ]:
data['DT'] = data['date']
data.set_index('DT', inplace=True)
data.head()

### 3) 날짜단위 지정하기 : freq

* **분석 단위**를 어떻게 가져갈 것인가와 관련이 있습니다.
* 시계열 데이터를 **일정한 시간 간격**으로 만들어 줍니다.
* 인덱스 조회시, 마지막에 있는 **freq** 옵션

In [ ]:
data.index

* 변경하기 
    * 시간단위 : H
    * 일 : D  (일 중 가장 빠른 시간의 데이터)
    * 월 : 
        * M (월 중 가장 마지막 날 데이터)
        * MS (월 중 가장 첫 날 데이터)    

* asfreq : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html
* freq 지정 방식 설명 : https://rfriend.tistory.com/506

In [ ]:
# 일단위
data.asfreq('D').head()

In [ ]:
# 월(말)단위
data.asfreq('M').head()

In [ ]:
# 월초 단위
data.asfreq('MS').head()

### 4) 빠진 값 찾기
예제 데이터에서는 없지만, 실제 쌓인 데이터에서는 빠진 데이터 발생 가능

In [ ]:
temp = data.asfreq('D')

In [ ]:
temp.isna().sum()

* 만약 빠진 데이터가 있다면, 아래와 같이 채울 수 있음.

In [ ]:
data.asfreq('D', method = 'ffill')

* 여기서는 일 단위 데이터이므로 D로 지정

In [ ]:
df = data.asfreq('D')
df.head()

## (2) y 만들기

* 사전 관찰(look-ahead) : 미래의 어떤 사실을 안다는 뜻
* 사전 관찰 문제 : 
    * 데이터를 통해 실제로 알아야 하는 시점보다 더 일찍 미래에 대한 사실을 알게 되는 문제.  
    * 사전관찰 문제가 있는 채로 모델링을 하게 되면, 놀라운 성능의 모델이 만들어짐. --> 그러나 실제로는 불가능한 상황.

* 그래서 y를 만들때 사전관찰문제가 발생되지 않도록 해야 함.
    * 예제는 1일 후의 수요량을 예측하려고 합니다.

* 1일 후 수요량을 예측하려면, y를 어떻게 만들어야 할까요?

In [ ]:
df['y'] = df['sales'].shift(-1)
display(df.head())
display(df.tail())

In [ ]:
# 제일 마지막 행은 삭제
df.dropna(axis = 0, inplace = True)
df.tail()

## (3) 데이터 분할

### 1) x, y 나누기

* .values(넘파이 어레이)로 변환해서 저장하는 이유 ➡ 데이터 스플릿 index를 적용해서 데이터를 가져오기 위해서

In [ ]:
target = 'y'

x = df.drop([target, 'date'], axis = 1)
y = df.loc[:, target]

### 2) 시계열 데이터 분할

* 다음의 조건으로 Cross Validation을 수행하겠습니다.
    * 3-fold
    * Validation 기간 30일

* TimeSeriesSplit : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [ ]:
x.iloc[:5]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
x.shape

In [ ]:
# validation set size
val_size = 30
nfold = 3

tscv = TimeSeriesSplit(n_splits = nfold, test_size = val_size)
tscv

In [ ]:
# .split을 이용하여 fold 하나씩 인덱스들을 뽑아 낼 수 있음.
for train_index, val_index in tscv.split(x):
    print("Train:", train_index, "Val:", val_index)

**tscv**는 모델링 수행시 cv 적용에 사용됩니다.

# 3.Baseline Model 생성

* Baseline 모델은 성능 시작점을 제공합니다.
* 성능 측정은 ML metric과 Biz metric으로 평가합니다.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import * 

## (1) loop 돌며 모델링(cross-validation) 수행

In [ ]:
rmse, mae, mape = [],[],[]
residuals = []
pred = []
model = LinearRegression()

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_val, y_val = x.iloc[val_index], y.iloc[val_index]

    # 학습
    model.fit(x_train, y_train)

    # 예측
    pr = model.predict(x_val)
    pred += list(pr)

    # 평가
    rmse.append(mean_squared_error(y_val, pr, squared = False))
    mae.append(mean_absolute_error(y_val, pr))
    mape.append(mean_absolute_percentage_error(y_val, pr))

    # 잔차 : 각 fold의 결과를 리스트로 변환하여 추가
    residuals += list(y_val - pr)

np.mean(rmse), np.mean(mae), np.mean(mape)

## (2) 결과 비교

In [ ]:
n = val_size * nfold

* pred를 시리즈로 바꾸고, 인덱스 맞추기

In [ ]:
pred = pd.Series(pred, index = y[-n:].index)
pred

* y_train, y_val(전체), pred 한꺼번에 시각화 

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y[:-n], label = 'train')
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

# 4.평가 : 잔차분석

* 잔차에 대한 우리의 기대 : 화이트 노이즈
    * 자기상관성 없음 : ACF, PACF 그래프
    * 정규분포 : Shapiro-Wilk 검정
    * 평균과 분산이 일정(Stationary) : ADF 검정
* 만약 화이트 노이즈가 아니라면...
    * 더 찾아내야 할 패턴이 있다는 의미.

## (1) 시각화

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(residuals)
plt.axhline(0, color = 'r', ls = '--')
plt.axhline(np.mean(residuals), color = 'g', ls = '--')
plt.show()

## (2) ACF, PACF

* 정상 데이터 및 자기상관관계가 없는 데이터라면
* ACF, PACF 그래프에서
    * 첫번째 lag 에서부터 하늘색 범위 안에 값이 위치해야 하고
    * 값의 등락에 대한 어떠한 패턴도 보이지 않아야 합니다.
* 그러나 실제 데이터에서는 그런 결과를 보기 쉽지 않습니다.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

### 1) ACF(자기 상관함수) 

In [ ]:
plot_acf(residuals, lags = 20)
plt.show()

In [ ]:
res = residuals

res_df = pd.DataFrame({'residuals':residuals})

for i in range(1,21) :
    var = 'lag' + str(i)
    res_df[var] = res_df['residuals'].shift(i)

In [ ]:
res_df.head(20)

In [ ]:
res_df.corr()

In [ ]:
plt.figure(figsize = (12,8))
sns.heatmap(res_df.corr(), annot = True)
plt.show()

### 2) PACF

In [ ]:
plot_pacf(residuals, lags = 20)
plt.show()

In [ ]:
lags = 20

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(residuals, lags = lags, ax = ax[0])
plot_pacf(residuals, lags = lags, ax = ax[1])
plt.show()

## (3) 검정
검정 도구를 적용하여 의사결정하는 용도로만 사용합니다.

In [ ]:
from scipy import stats
import statsmodels.api as sm

### 1) 정규성 검정 : Shapiro-Wilk 검정

* 귀무가설 : **정규 분포이다.** (p-value > 0.05)
* 대립가설 : 정규분포가 아니다.

In [ ]:
stats.shapiro(residuals)[1]

### 2) 정상성 검정 : ADF 검정

* 귀무가설 : 비정상(Non-Stationary) 데이터이다.
* 대립가설 : **정상(Stationary) 데이터**이다.(P-value <= 0.05)

In [ ]:
# ADF 테스트
sm.tsa.stattools.adfuller(residuals)[1]